# Goal
- To compare distance metrics suitable for the firefly optimization

# Table of Contents
- [Imports](#imports)
- [Preprocessing](#preprocessing)
- [Split data](#split-data)
- [Basic CatBoost Model](#basic-catboost-model)
- [Firefly Algorithm Definition](#firefly-algorithm-definition)
- [Distance Metric Selection Obtained](#best-model-hyperparameters-obtained)


# Imports

In [7]:
# Data Handling
import pandas as pd
import numpy as np
import os

# Machine Learning & Optimization
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utility
import math


In [3]:
def score(actual_values, predicted_values):
    mae_score = mean_absolute_error(actual_values, predicted_values)
    R2_score = r2_score(actual_values,predicted_values)
    RMSE = math.sqrt(mean_squared_error(actual_values,predicted_values))
    return {"MAE":mae_score, "R2_score":R2_score, "RMSE":RMSE}



# Preprocessing

In [2]:
# Step 2: Load dataset
data_filepath = r"Wiley.csv"

data = pd.read_csv(data_filepath)
# remove
data = data.drop(['Index','Unnamed: 16'], axis=1)
print(data.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 785 entries, 0 to 784
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Cement (kg/m3)  785 non-null    float64
 1   SF (kg/m3)      785 non-null    float64
 2   BFS (kg/m3)     785 non-null    float64
 3   FA (kg/m3)      785 non-null    float64
 4   QP (kg/m3)      785 non-null    float64
 5   LSP (kg/m3)     785 non-null    float64
 6   NS (kg/m3)      785 non-null    float64
 7   Fiber (kg/m3)   785 non-null    float64
 8   Sand (kg/m3)    785 non-null    float64
 9   Gravel (kg/m3)  785 non-null    int64  
 10  Water (kg/m3)   785 non-null    float64
 11  SP (kg/m3)      785 non-null    float64
 12  T (oC)          785 non-null    int64  
 13  RH (%)          785 non-null    int64  
 14  Age (days)      785 non-null    int64  
 15  fc (MPa)        785 non-null    float64
dtypes: float64(12), int64(4)
memory usage: 98.3 KB
None


### Split data

In [4]:
X = data.drop('fc (MPa)', axis=1)  # Replace 'fc (MPa)' with your target column name
y = data['fc (MPa)']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Basic Catboost model

In [5]:
basic_model = CatBoostRegressor(verbose=0)
basic_model.fit(X_train,y_train)

basic_model_predictions = basic_model.predict(X_valid)
print(score(y_valid, basic_model_predictions))

{'MAE': 4.842734419072415, 'R2_score': 0.9737849671809723, 'RMSE': 6.407226590840827}


In [6]:
basic_model_predictions_train = basic_model.predict(X_train)
print(score(y_train, basic_model_predictions_train))

{'MAE': 2.738650005987045, 'R2_score': 0.9899561251353487, 'RMSE': 4.031897368641099}


# Firefly algorithm definition

- Five distance metrics are used for comparison
    - Euclidean Distance 
    - Manhattan Distance (L1 norm)
    - Minkowski Distance (generalized)
    - Cosine Distance
    - Chebyshev Distance

In [13]:
def initialize_fireflies(population_size, dimension):
  fireflies = np.zeros((population_size, dimension))
  for i in range(population_size):
    for j in range(dimension):
      fireflies[i][j] = np.random.uniform(param_bounds[j][0],param_bounds[j][1])
      # make sure that depth, border_count and iterations are integers
      if j == 1 or j==5 or j==6:
        fireflies[i][j] = round(fireflies[i][j])
  return fireflies

def objective_function(solution,to_return=1):

    learning_rate = solution[0]
    if learning_rate < 0.01: 
        learning_rate = 0.01
    elif learning_rate > 0.05:
        learning_rate = 0.05

    depth = int(solution[1])
    if depth < 4: 
        depth = 4
    elif depth > 10:
        depth = 10

    l2_leaf_reg = solution[2]
    if l2_leaf_reg < 0.01: 
        l2_leaf_reg = 0.01
    elif l2_leaf_reg > 100:
        l2_leaf_reg = 100

    bagging_temperature = solution[3]
    if bagging_temperature < 0: 
        bagging_temperature = 0
    elif bagging_temperature > 10:
        bagging_temperature = 10

    subsample = solution[4]
    if subsample < 0.5: 
        subsample = 0.5
    elif subsample > 0.8:
        subsample = 0.8

    border_count = int(solution[5])
    if border_count < 1:
        border_count = 1
    elif border_count > 255:
        border_count = 255

    iterations = int(solution[6])
    if iterations < 100:
        iterations = 100
    elif iterations > 1500:
        iterations = 1500
    
    model = CatBoostRegressor(
        learning_rate=learning_rate,
        depth=depth,
        l2_leaf_reg=l2_leaf_reg,
        subsample=subsample,
        border_count=border_count,
        iterations=iterations,
        verbose=0
    )

    # Fit the model on the training data with early stopping based on validation data
    model.fit(X_train,y_train)

    # Make predictions on the validation data
    y_pred = model.predict(X_valid)

    if to_return==1:
    # Calculate and print the performance using mean squared error (MSE)
        MSE = mean_squared_error(y_valid,y_pred)
        return MSE
    else:
        return score(y_valid,y_pred)

def calculate_intensity(population, objective_func):
    return np.array([objective_func(ind) for ind in population])

def calculate_distance_euclidean(firefly_i, firefly_j):
    # Standard scale each dimension
    firefly_i_scaled = (firefly_i - means) / stds
    firefly_j_scaled = (firefly_j - means) / stds

    # Calculate Euclidean distance after standard scaling
    distance = np.linalg.norm(firefly_i_scaled - firefly_j_scaled)
    return distance

def calculate_distance_manhattan(firefly_i, firefly_j):
    firefly_i_scaled = (firefly_i - means) / stds
    firefly_j_scaled = (firefly_j - means) / stds
    distance = np.sum(np.abs(firefly_i_scaled - firefly_j_scaled))
    return distance

def calculate_distance_cosine(firefly_i, firefly_j):
    firefly_i_scaled = (firefly_i - means) / stds
    firefly_j_scaled = (firefly_j - means) / stds
    dot_product = np.dot(firefly_i_scaled, firefly_j_scaled)
    norm_i = np.linalg.norm(firefly_i_scaled)
    norm_j = np.linalg.norm(firefly_j_scaled)
    cosine_similarity = dot_product / (norm_i * norm_j + 1e-8)
    distance = 1 - cosine_similarity
    return distance

def calculate_distance_chebyshev(firefly_i, firefly_j):
    firefly_i_scaled = (firefly_i - means) / stds
    firefly_j_scaled = (firefly_j - means) / stds
    distance = np.max(np.abs(firefly_i_scaled - firefly_j_scaled))
    return distance

def calculate_distance_canberra(firefly_i, firefly_j):
    firefly_i_scaled = (firefly_i - means) / stds
    firefly_j_scaled = (firefly_j - means) / stds
    numerator = np.abs(firefly_i_scaled - firefly_j_scaled)
    denominator = np.abs(firefly_i_scaled) + np.abs(firefly_j_scaled) + 1e-8
    distance = np.sum(numerator / denominator)
    return distance


def calculate_attractiveness(distance_ij, beta_0, gamma):
    return beta_0 * np.exp(-gamma * distance_ij**2)

def save_population_and_intensity_to_csv(population, intensity, iteration,distance_function):
    # Create a DataFrame from the population and intensity
    df = pd.DataFrame(population, columns=[f'Param_{i+1}' for i in range(population.shape[1])])
    df.insert(0, 'Intensity', intensity)
    df.to_csv(f'output2/fireflies_population_intensity_iteration_{distance_function}_{iteration}.csv', index=False)

def move_fireflies(population, intensity, max_iter, alpha, beta_0, gamma,distance_function):
    population_size, dimension = population.shape

    for iter_num in range(max_iter):
        for i in range(population_size):
            for j in range(population_size):
                if intensity[i] > intensity[j]:
                    distance_ij = distance_function(population[i], population[j])
                    attractiveness_ij = calculate_attractiveness(distance_ij, beta_0, gamma)
                    population[i] += attractiveness_ij * (population[j] - population[i]) + \
                                     alpha * (np.random.rand(dimension) - 0.5)
        
        intensity = calculate_intensity(population, objective_function)
        save_population_and_intensity_to_csv(population, intensity, iter_num,distance_function.__name__)
        
    best_solution = population[np.argmin(intensity)]
    return best_solution


In [14]:
# param_bounds = {
#       'learning_rate': [0.01, 0.05],  # Using the refined range for learning rate
#       'depth': [4,10],  # Refined range for tree depth
#       'l2_leaf_reg': [0.01, 100],  # L2 regularization range
#       'bagging_temperature': [0, 10],  # Bagging temperature range
#       'subsample': [0.5, 0.8],  # Subsample range
#       'border_count': [1, 255],  # Feature binning precision range
#       'iterations': [100, 1500]  # Number of iterations for training
#   }
# the ranges are modified a bit to adjust for the rounding later on
# e.g depth is 0.6 to 16.4 in order to adjust for rounding
param_bounds = [ [0.01, 0.05], [4, 10], [0.01, 100], [0, 10], [0.5, 0.8], [0.7, 255.3], [100, 1500.9]]

population_size = 10
dimension = 7
max_iter = 30
alpha = 0.1
beta_0 = 1.0
gamma = 1.0

means = np.array([(bounds[0] + bounds[1]) / 2 for bounds in param_bounds])
stds = np.array([(bounds[1] - bounds[0]) / 4 for bounds in param_bounds])  # Dividing by 4 assumes a reasonable spread


In [17]:
fireflies = initialize_fireflies(population_size, dimension)

intensity = calculate_intensity(fireflies, objective_function)


distance_functions = [calculate_distance_euclidean,calculate_distance_manhattan,calculate_distance_cosine,calculate_distance_chebyshev,calculate_distance_canberra]
for distance_function in distance_functions:
    best_solution = move_fireflies(fireflies, intensity, max_iter, alpha, beta_0, gamma,distance_function)

    print(f"Best Solution: {distance_function.__name__}", best_solution)
    print("Objective Value:", objective_function(best_solution))

Best Solution: calculate_distance_euclidean [6.00706515e-02 8.99413794e+00 2.09190268e+01 9.12879361e+00
 6.74832053e-01 2.29010432e+02 1.19799900e+03]
Objective Value: 42.40286739307526
Best Solution: calculate_distance_manhattan [7.17355195e-02 8.97473383e+00 2.09079947e+01 9.20116962e+00
 6.81926411e-01 2.29012307e+02 1.19795396e+03]
Objective Value: 43.874030173455694
Best Solution: calculate_distance_cosine [1.24882429e-01 7.71214000e+00 6.44533968e+00 7.78916401e+00
 3.57211685e-02 1.18030578e+02 5.60154359e+02]
Objective Value: 43.013890520915446
Best Solution: calculate_distance_chebyshev [1.03726445e-01 8.01279208e+00 6.52226278e+00 7.54970223e+00
 1.18927522e-01 1.18124517e+02 5.59726763e+02]
Objective Value: 42.95963948917333
Best Solution: calculate_distance_canberra [ 9.45174406e-02  7.78861894e+00  6.44577604e+00  7.87597603e+00
 -2.73300171e-01  1.17895314e+02  5.60229976e+02]
Objective Value: 43.04540434956417


# Distance Metric Selection

- sort solutions for each distance metric
- select best from each distance metric
- compare
- Result
    - Euclidean distance is chosen based on comparison of three metrics as it tops in R2 and RMSE

In [17]:
# Specify the folder path
folder_path = 'output2'

# List all files in the folder
files = os.listdir(folder_path)
distance_metrics = {'manhattan': [], 'chebyshev': [], 'canberra': [], 'euclidean': [], 'cosine': []}

# Iterate through each file
for file_name in files:
    file_path = os.path.join(folder_path, file_name)


    # Open and read the file (assuming text files)
    with open(file_path, 'r', encoding='utf-8') as file:
        df_file = pd.read_csv(file_path)
        df_sorted = df_file.sort_values(by='Intensity')
        distance_metric = file_name.split("_")[6]
        distance_metrics[distance_metric].append(df_sorted.head(10).copy())

for distance_metric in distance_metrics:
    df_result = pd.concat(distance_metrics[distance_metric], ignore_index=True)
    df_result = df_result.sort_values(by='Intensity')
    df_10 = df_result.head(1)
    print(f"Solutions for {distance_metric}:\n")
    for i in range(len(df_10)):
        solution = list(df_10.iloc[i,1:])
        print(f"solution: {solution}")
        print(objective_function(solution,2))
        print()


# fireflies_population_intensity_iteration_calculate_distance_manhattan_29

Solutions for manhattan:

solution: [0.0717355195207518, 8.974733834315613, 20.907994688106324, 9.201169615336676, 0.6819264111815221, 229.0123068221245, 1197.953960655442]
{'MAE': 4.8924301078234995, 'R2_score': 0.9719832490688581, 'RMSE': 6.623747441853116}

Solutions for chebyshev:

solution: [0.1037264449085622, 8.012792077043219, 6.522262782635443, 7.549702232940268, 0.1189275220528532, 118.12451699547547, 559.7267630901813]
{'MAE': 4.909590281483083, 'R2_score': 0.9725671538515738, 'RMSE': 6.554360341724685}

Solutions for canberra:

solution: [0.094517440641604, 7.788618939381601, 6.445776040754938, 7.87597603496103, -0.2733001707195872, 117.89531392511672, 560.2299758141733]
{'MAE': 4.851880051563085, 'R2_score': 0.9725123867667468, 'RMSE': 6.5608996600743845}

Solutions for euclidean:

solution: [0.0600706514619212, 8.994137940297726, 20.91902678932367, 9.128793605952286, 0.6748320532853389, 229.0104322400013, 1197.9990025378909]
{'MAE': 4.889092636468801, 'R2_score': 0.972922